In [1]:
!pip install yfinance

     |████████████████████████████████| 6.4 MB 36.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import numpy as np
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from math import sqrt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
import keras.backend as K

In [3]:
def get_historical_klines_with_efficiancy_rate(symbol, start, interval):
  ticker = yf.Ticker(symbol)
  df = ticker.history(start=start,interval=interval)
  df.drop(['Dividends','Stock Splits'], axis=1, inplace=True)
  df['efficiency_rate'] = ((df['Close']-df['Open'])/df['Open']) * 100
  return df

In [4]:
start="2017-01-01"
interval="1d"
btc_df = get_historical_klines_with_efficiancy_rate("BTC-USD", start, interval)

kmeans = KMeans(n_clusters=3)
btc_df["Cluster"] = kmeans.fit_predict(btc_df[['efficiency_rate']])

clusters = kmeans.cluster_centers_.argsort(axis=0).flatten().tolist()

btc_df["Cluster"] = btc_df["Cluster"].apply(lambda x: clusters.index(x)-1)
btc_df["Cluster"] = btc_df["Cluster"].astype("category")

btc_df

,Open,High,Low,Close,Volume,efficiency_rate,Cluster
Date,,,,,,,
2017-01-01,963.658020,1003.080017,958.698975,998.325012,147775008,3.597437,1
2017-01-02,998.617004,1031.390015,996.702026,1021.750000,222184992,2.316503,0
2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,185168000,2.176976,0
2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,344945984,10.563956,1
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,510199008,-12.392691,-1
...,...,...,...,...,...,...,...
2022-05-01,37713.265625,38627.859375,37585.789062,38469.093750,27002760110,2.004144,0
2022-05-02,38472.187500,39074.972656,38156.562500,38529.328125,32922642426,0.148525,0
2022-05-03,38528.109375,38629.996094,37585.621094,37750.453125,27326943244,-2.018413,0


In [5]:
btc_df['lag_1'] = btc_df['efficiency_rate'].shift(1)
btc_df['lag_2'] = btc_df['efficiency_rate'].shift(2)
btc_df['lag_3'] = btc_df['efficiency_rate'].shift(3)
btc_df['lag_4'] = btc_df['efficiency_rate'].shift(4)
btc_df.dropna(inplace=True)
btc_df

,Open,High,Low,Close,Volume,efficiency_rate,Cluster,lag_1,lag_2,lag_3,lag_4
Date,,,,,,,,,,,
2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,510199008,-12.392691,-1,10.563956,2.176976,2.316503,3.597437
2017-01-06,1014.239990,1046.810059,883.943970,902.200989,351876000,-11.046597,-1,-12.392691,10.563956,2.176976,2.316503
2017-01-07,903.487000,908.585022,823.556030,908.585022,279550016,0.564261,0,-11.046597,-12.392691,10.563956,2.176976
2017-01-08,908.174988,942.723999,887.249023,911.198975,158715008,0.332974,0,0.564261,-11.046597,-12.392691,10.563956
2017-01-09,913.244019,913.685974,879.807007,902.828003,141876992,-1.140551,0,0.332974,0.564261,-11.046597,-12.392691
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,37713.265625,38627.859375,37585.789062,38469.093750,27002760110,2.004144,0,-2.307899,-2.913838,1.356725,2.940219
2022-05-02,38472.187500,39074.972656,38156.562500,38529.328125,32922642426,0.148525,0,2.004144,-2.307899,-2.913838,1.356725
2022-05-03,38528.109375,38629.996094,37585.621094,37750.453125,27326943244,-2.018413,0,0.148525,2.004144,-2.307899,-2.913838


In [6]:
cols_to_use = []
for i in range(1,5):
  cols_to_use.append('lag_{}'.format(i))

X = btc_df[cols_to_use]
y = btc_df.efficiency_rate

In [7]:
negative_train_samples, zero_train_samples, positive_train_samples =  btc_df[btc_df['Cluster']==-1].sample(250), btc_df[btc_df['Cluster']==0].sample(250), btc_df[btc_df['Cluster']==1].sample(250)
all_train_samples = pd.concat([negative_train_samples, zero_train_samples, positive_train_samples])
X_train = all_train_samples[cols_to_use]
y_train = all_train_samples.efficiency_rate

In [8]:
btc_df = btc_df.drop(negative_train_samples.index)
btc_df = btc_df.drop(zero_train_samples.index)
btc_df = btc_df.drop(positive_train_samples.index)

In [9]:
negative_test_samples, zero_test_samples, positive_test_samples =  btc_df[btc_df['Cluster']==-1].sample(50), btc_df[btc_df['Cluster']==0].sample(50), btc_df[btc_df['Cluster']==1].sample(50)
all_test_samples = pd.concat([negative_test_samples, zero_test_samples, positive_test_samples])
X_test= all_test_samples[cols_to_use]
y_test= all_test_samples.efficiency_rate

In [40]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

def custom_loss_function(y_true, y_pred):
   return K.sum(K.abs((y_pred - y_true) * y_true))

regressor.compile(loss=custom_loss_function, optimizer='adam')

regressor.fit(X_train, y_train, epochs = 200, batch_size = 32)

Epoch 1/200
24/24 [==============================] - 9s 13ms/step - loss: 1008.4072
Epoch 2/200
24/24 [==============================] - 0s 13ms/step - loss: 1007.0464
Epoch 3/200
24/24 [==============================] - 0s 14ms/step - loss: 1004.0629
Epoch 4/200
24/24 [==============================] - 0s 14ms/step - loss: 999.3607
Epoch 5/200
24/24 [==============================] - 0s 13ms/step - loss: 994.6376
Epoch 6/200
24/24 [==============================] - 0s 14ms/step - loss: 996.4439
Epoch 7/200
24/24 [==============================] - 0s 14ms/step - loss: 982.4777
Epoch 8/200
24/24 [==============================] - 0s 15ms/step - loss: 977.7878
Epoch 9/200
24/24 [==============================] - 0s 13ms/step - loss: 979.6179
Epoch 10/200
24/24 [==============================] - 0s 14ms/step - loss: 970.1744
Epoch 11/200
24/24 [==============================] - 0s 14ms/step - loss: 974.6686
Epoch 12/200
24/24 [==============================] - 0s 14ms/step - loss: 970.731

# Confusion matrix for test data for three clusters -1 0 1
the order is exact same ( -1  0  1)

In [41]:
preds = regressor.predict(X_test)
preds = preds.astype('float64')
y_preds = kmeans.predict(preds)
for i in range(len(y_preds)):
  y_preds[i] = clusters.index(y_preds[i]) -1
y_test_clusters = kmeans.predict(np.array(y_test).reshape(-1, 1))
for i in range(len(y_test_clusters)):
  y_test_clusters[i] = clusters.index(y_test_clusters[i]) -1
confusion_matrix(y_test_clusters, y_preds)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  "X does not have valid feature names, but"


array([[21, 16, 13],
       [17, 10, 23],
       [10, 17, 23]])

# Confusion matrix for train data for three clusters -1 0 1

In [42]:
preds = regressor.predict(X_train)
preds = preds.astype('float64')
y_preds = kmeans.predict(preds)
for i in range(len(y_preds)):
  y_preds[i] = clusters.index(y_preds[i]) -1
y_train_clusters = kmeans.predict(np.array(y_train).reshape(-1, 1))
for i in range(len(y_train_clusters)):
  y_train_clusters[i] = clusters.index(y_train_clusters[i]) -1
confusion_matrix(y_train_clusters, y_preds)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  "X does not have valid feature names, but"


array([[135,  57,  58],
       [ 48,  87, 115],
       [ 25,  66, 159]])